# <center>Multi-View Image Classification</center>

In [ ]:
#=============================================================================================================#
#                                                   IMPORTS                                                   #
#=============================================================================================================#

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
import os
import time
from PIL import Image
import cv2
import csv
import copy

from collections import OrderedDict
from scipy import spatial
import glob

from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, StratifiedShuffleSplit
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import make_scorer, accuracy_score

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

from torchvision import models
import torchvision.transforms as transforms


import pickle
from joblib import dump, load

import warnings
warnings.filterwarnings('ignore')


In [ ]:
#======================================================================================================================#
#                                           UTILITY FUNCTIONS & GLOBALS                                                #
#======================================================================================================================#

def compute_sample_weight(class_weights, y):
    """
    Generate a weight array for the array y

    :param class_weights: (type dict)
        weights of the classes in y
    :param y: (type iterable)
        array for which the weight array has to be generated
    :return: (type numpy array)
        weight array
    """

    assert isinstance(class_weights, dict)
    result = np.array([class_weights[i] for i in y])
    return result

# Class Weights used in the weighted accuracy
class_weights = {
    0: 0.327,
    2: 2.0,
    3: 2.572,
    1: 0.586,
    5: 15.0,
    4: 26.452}


## I. Data Processing

In [ ]:
# GLOBALS

# Data Directory
data_dir = '../data/raw'

# Image Processing Parameters 
BLUR = 21
CANNY_THRESH_1 = 10
CANNY_THRESH_2 = 200
MASK_DILATE_ITER = 10
MASK_ERODE_ITER = 10
MASK_COLOR = (0.0, 0.0, 1.0) # In BGR format

In [ ]:
# IMAGE PREPROCESSING FUNCTION

def preprocess(filename):
    img = cv2.imread(filename)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    #-- Edge detection -------------------------------------------------------------------
    edges = cv2.Canny(gray, CANNY_THRESH_1, CANNY_THRESH_2)
    edges = cv2.dilate(edges, None)
    edges = cv2.erode(edges, None)
    
    #-- Find contours in edges, sort by area ---------------------------------------------
    contour_info = []
    _, contours, _ = cv2.findContours(edges, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)

    for c in contours:
        contour_info.append((
            c,
            cv2.isContourConvex(c),
            cv2.contourArea(c),
        ))
    contour_info = sorted(contour_info, key=lambda c: c[2], reverse=True)
    max_contour = contour_info[0]
    
    #-- Create empty mask, draw filled polygon on it corresponding to largest contour ----
    # Mask is black, polygon is white
    mask = np.zeros(edges.shape)
    cv2.fillConvexPoly(mask, max_contour[0], (255))

    #-- Smooth mask, then blur it --------------------------------------------------------
    mask = cv2.dilate(mask, None, iterations=MASK_DILATE_ITER)
    mask = cv2.erode(mask, None, iterations=MASK_ERODE_ITER)
    mask = cv2.GaussianBlur(mask, (BLUR, BLUR), 0)

    #-- Create final image ---------------------------------------------------------------
    img[mask <= 100] = 0
    return img
 

In [ ]:
# Preprocessing an example image
filename = '../data/raw/original_2.png'
img = preprocess(filename)

plt.imshow(img);

In [ ]:
# Preprocess all images and Save them to a new directory `processed`

filenames = glob.glob(os.path.join(data_dir, '*.png')) # list all image file names in the data folder
proc_data_dir = '../data/processed'

for fname in filenames:
    img_name = os.path.basename(fname)
    img = preprocess(fname)
    cv2.imwrite(os.path.join(proc_data_dir, os.path.splitext(img_name)[0] + '_processed.png'), img)

In [ ]:
# Function for visualizing an image

def show_plug(plug_name, data_dir, preprocess=None):
    # Read Images
    _fnames = glob.glob(os.path.join(data_dir, f'{_name}_*.png'))
    if preprocess:
        _images = [preprocess(_fname) for _fname in _fnames]
    else:
        _images = [cv2.cvtColor(cv2.imread(_fname), cv2.COLOR_BGR2RGB) for _fname in _fnames]
    # Visualize Images
    plt.figure(figsize=(18,8))
    for i in range(len(_images)):
        plt.subplot(2, 4, i+1)
        plt.imshow(_images[i])
        plt.title(os.path.basename(_fnames[i]))
        plt.xticks([])
        plt.yticks([])
        
    plt.suptitle(f'IMAGE: {_name}', size=20)
    
    


In [ ]:
# Example Visualization

_name = 'original_2'
show_plug(plug_name, data_dir)
show_plug(plug_name, data_dir, preprocess=preprocess)

## II. Feature Extraction & Machine Learning

### 1. Build Vocabulary of Visual Words

The first step is building a vocabulary of visual words. This task relies on one or a combination of multiple traditional computer vision feature extractors such as : HoG, SURF, SIFT etc. Let us first use SURF to extract features from an example image, visualize the key-points identified, then extract features from all of the images into a matrix of descriptors. We then use K-means on this matrix to come up with our K visual words (represented by the K centroids).

In [ ]:
# Detect and Visualize Keypoints in an Image

# Choosing a high value for the hessian threshold in order to visualize only a few key-points to avoid clutter
surf = cv2.xfeatures2d.SURF_create(2000) 

img = cv2.imread('../data/processed/original_2_13_processed.png')
img = cv2.resize(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY), (900, 900))

kp, descriptors = surf.detectAndCompute(img, None)
print('The number of key points detected:', len(kp))

img_kp = cv2.drawKeypoints(img, kp, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
plt.figure(figsize=(5, 5))
plt.imshow(img_kp, cmap='gray')
plt.xticks([])
plt.yticks([]);

In [ ]:
# CREATE THE MATRIX OF DESCRIPTORS FROM ALL IMAGES

proc_data_dir = '../data/processed'
filenames = glob.glob(os.path.join(proc_data_dir, '*.png')) # list all image file names in the data folder

surf = cv2.xfeatures2d.SURF_create(300) # create SURF feature extractor


counter = 0
list_descriptors = []

for filename in filenames:
    img = cv2.imread(filename)
    img = cv2.resize(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY), (300, 300))
    kp, descriptors = surf.detectAndCompute(img, None)
    if descriptors is not None:
        list_descriptors.append(descriptors)

# This is a matrix where each row is a feature vector extracted from an image (possible to have multiple features per image)      
# And each column is a dimension of the SURF feature vector (total 64, can also be 128 which can be set in the definition of surf) 
descriptor_matrix = np.vstack(list_descriptors)
print('Shape of the Matrix of Descriptors', descriptor_matrix.shape)

In [ ]:
# CLUSTER FEATURE VECTORS TO CONSTRUCT THE CODEBOOK (i.e VOCABULARY OF VISUAL WORDS)
# Training Kmeans takes time, we save it at the end so we can reload it again later
# Note : bigger K (eg. 500 or 800) will yield better results at the expense of more computation time
K = 200
km = KMeans(n_clusters=K, n_jobs=-1)
km.fit(descriptor_matrix)
centroids = km.cluster_centers_
print('The shape of the centroids:', centroids.shape)

del descriptor_matrix # To free up memory

# Save the k-means model and its centroids
#np.save('../resources/vocabulary_surf.npy', centroids)
#pickle.dump(km, open('../models/kmeans.pkl', 'wb'))

### 2. Build Matrix of Feature Vectors

Here, we build the matrix of feature vectors by transforming each image into a vector of size K (number of clusters from before). For each we process all its image views, extract their SURF descriptors, find the cluster of each descriptor (i.e. the visual word associated with the descriptor), then count the number of times each visual word (i.e. cluster) occurs. We end up with a feature vector of size K for that Cimage. Stacking these feature vectors vertically results in a NxK matrix where each row represents an image and each column, a visual word and each cell, the number of times the visual word occurs in the image (i.e all of its image views).

In [ ]:
# FUNCTION DEFINITION : turns an image set into a feature vector of size K through image quantization.
# We use the K-means model trained on the matrix of descriptors
# Iterate through the images, process each image, create its SURF feature vectors
# Predict the cluster of each feature vector, add 1 to the output vector at the location of the predicted cluster (Quantization)

def get_feature_vec(plug_name, model, preprocess=None, surf_threshold=200):
    
    feature_vec = np.zeros(K) # Intialize vector representation of visual words
    
    surf = cv2.xfeatures2d.SURF_create(surf_threshold)
    
    _fnames = glob.glob(os.path.join(proc_data_dir, f'{_name}_*.png'))
    if preprocess:
        _images = [cv2.resize(cv2.cvtColor(preprocess(_fname), cv2.COLOR_BGR2GRAY), (300, 300)) for plug_fname in _fnames] 
    else:
        _images = [cv2.resize(cv2.cvtColor(cv2.imread(_fname), cv2.COLOR_BGR2GRAY), (300, 300)) for plug_fname in _fnames] 
    
    for image in _images: # iterate over the image views of the image
        kp, descriptors = surf.detectAndCompute(image, None) # extract the surf descriptors of the image
        if descriptors is not None:
            visual_words = model.predict(descriptors) # find all the visual words in the image
            for visual_word in visual_words: # find the occurence of each visual word
                feature_vec[visual_word] += 1

    return feature_vec


In [ ]:
# Example : Visualizing the feature vector of two images as a bar graph

_name_1 = 'original_2'
_name_2 = 'original_3'

_fname_1 = os.path.join(proc_data_dir, f'{plug_name_1}_1_processed.png')
_fname_2 = os.path.join(proc_data_dir, f'{plug_name_2}_1_processed.png')

img_1 = cv2.cvtColor(cv2.imread(plug_fname_1), cv2.COLOR_BGR2RGB)
img_2 = cv2.cvtColor(cv2.imread(plug_fname_2), cv2.COLOR_BGR2RGB)


kp_1 = surf.detect(img_1)
kp_2 = surf.detect(img_2)

plt.figure(figsize=(18,10))

plt.subplot(2, 3, 1)
plt.imshow(img_1)
plt.title('Image view 1')
plt.xticks([]); plt.yticks([]) 
plt.subplot(2, 3, 2)
plt.imshow(cv2.drawKeypoints(cv2.cvtColor(img_1, cv2.COLOR_RGB2GRAY), kp_1, None, flags=cv2.DrawMatchesFlags_DRAW_RICH_KEYPOINTS))
plt.title('Keypoints detected')
plt.xticks([]); plt.yticks([]) 
plt.subplot(2, 3, 3)
plt.bar(np.arange(1, K+1), get_feature_vec(_name_1, km), color='mediumaquamarine', width=0.3)
plt.title('Feature Vector')

plt.subplot(2, 3, 4)
plt.imshow(img_2)
plt.title('Image view 2')
plt.xticks([]); plt.yticks([]) 
plt.subplot(2, 3, 5)
plt.imshow(cv2.drawKeypoints(cv2.cvtColor(img_2, cv2.COLOR_RGB2GRAY), kp_2, None, flags=cv2.DrawMatchesFlags_DRAW_RICH_KEYPOINTS))
plt.title('Keypoints detected')
plt.xticks([]); plt.yticks([]) 
plt.subplot(2, 3, 6)
plt.bar(np.arange(1, K+1), get_feature_vec(_name_2, km), color='mediumaquamarine', width=0.3)
plt.title('Feature Vector of images');

In [ ]:
# BUILD THE MATRIX OF FEATURE VECTORS (NxK)

_names = np.unique([fname.split('_')[0] for fname in os.listdir(proc_data_dir) if fname.endswith('.png')])
N = len(_names)
print(f'There are {N} images.')

X = np.empty((N, K))

for i, _name in enumerate(_names):
    X[i] = get_feature_vec(_name, km)

print('Shape of the feature matrix', X.shape)

### 3. Train a Machine Learning Model

In [ ]:
# GET CAR PLUG LABELS
df_class_map = pd.read_csv('../data/processed/train.csv')
df_class_map.head()

labels = [df_class_map.loc[df_class_map.part_no == _name, 'label'].item() for _name in _names]
labels[:10]

y = np.array([class_id_map[label] for label in labels]) # Encode the string labels into code numbers
y[:10]

In [ ]:
# TRAIN-TEST SPLIT
rnd = 42
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=rnd)

In [ ]:
# NORMALIZE DATA
m = X_train.mean(axis=0)
std = X_train.std(axis=0)

X_train = (X_train - m)/std
X_test = (X_test - m)/std

# Need to save mean and std of training data, we use these values to normalize test data and new data as well
#np.save('../resources/scaler_mean', m)
#np.save('../resources/scaler_std', std)

In [ ]:
# TRAIN A LOGISTIC REGRESSION MODEL 
# We using cost-sensitive learning by providing the class weights to steer the learning towards making less costly mistakes as represented by the weights

lr = LogisticRegression(class_weight=class_weights)
lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)
accuracy = accuracy_score(y_test, 
                          y_pred, 
                          sample_weight=compute_sample_weight(class_weights, y_test))

print(f'The accuracy of Logistic Regression is {round(accuracy*100, 2)}%')

In [ ]:
# SAVE THE MODEL
#pickle.dump(lr, open('../models/logistic_regression.pkl', 'wb'))

**Comment :**

Although the accuracy seems high, it has a very large variance. If we resample the training and test set again, we may obtain a very different performance value. The values typically range between 76% and 94% with an average around 87%.

In [ ]:
# FUNCTION TO GET PREDICTIONS FOR IMAGES

def lr_pred(_name, data_dir, pred_model, clus_model, scaler_mean, scaler_std):

    feature_vec = get_feature_vec(_name, clus_model, preprocess=preprocess)
    feature_vec = (feature_vec - scaler_mean) / scaler_std
    pred = pred_model.predict(feature_vec.reshape(1, -1))[0]
    return pred, {v:k for k,v in class_id_map.items()}[pred]


In [ ]:
# GET NEW PREDICTIONS FROM RAW IMAGES
_name = 'original_6'
lr_pred(_name, data_dir, lr, km, m, std)

## III. Multi-View Convolutional Neural Network (MVCNN)

In [ ]:
# CREATE STRATIFIED TRAIN-VALIDATION SPLIT INDICES
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2)
y = [dataset._label_map[image] for image in dataset._names]

train_indices, val_indices = next(sss.split(np.zeros(len(y)), y))

In [ ]:
# CREATE TRAIN AND VALIDATION DATA LOADERS
train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)

train_loader = DataLoader(dataset, batch_size=32, sampler=train_sampler, num_workers=8)
val_loader = DataLoader(dataset, batch_size=32, sampler=val_sampler, num_workers=8)
data_loaders = {'train': train_loader, 'val': val_loader}

## 2. Create the Multi-View CNN (MVCNN) Architecture 

In [ ]:
# MULTI-VIEW CONVOLUTIONAL NEURAL NETWORK (MVCNN) ARCHITECTURE

class MVCNN(nn.Module):
    def __init__(self, num_classes=1000, pretrained=True):
        super(MVCNN, self).__init__()
        resnet = models.resnet34(pretrained = pretrained)
        fc_in_features = resnet.fc.in_features
        self.features = nn.Sequential(*list(resnet.children())[:-1])
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(fc_in_features, 2048),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(2048, 2048),
            nn.ReLU(inplace=True),
            nn.Linear(2048, num_classes)
        )

    def forward(self, inputs): # inputs.shape = samples x views x height x width x channels
        inputs = inputs.transpose(0, 1)
        view_features = [] 
        for view_batch in inputs:
            view_batch = self.features(view_batch)
            view_batch = view_batch.view(view_batch.shape[0], view_batch.shape[1:].numel())
            view_features.append(view_batch)   
            
        pooled_views, _ = torch.max(torch.stack(view_features), 0)
        outputs = self.classifier(pooled_views)
        return outputs

In [ ]:
# BUILD AND VISUALIZE THE MODEL
model = MVCNN(num_classes=6, pretrained=True)
print(model)

## 3. Train the MVCNN Model

In [ ]:
# DEFINE THE DEVICE
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

In [ ]:
# DEFINE A FUNCTION TO TRAIN THE MODEL

def train_model(model, dataloaders, criterion, optimizer, num_epochs=25):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(1, num_epochs+1):
        print('Epoch {}/{}'.format(epoch, num_epochs))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0
            all_preds = []
            all_labels = []
            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    # Get model predictions
                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                all_preds.append(preds)
                all_labels.append(labels)

            epoch_loss = running_loss / len(dataloaders[phase].sampler.indices)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].sampler.indices)
            all_labels = torch.cat(all_labels, 0)
            all_preds = torch.cat(all_preds, 0)
            epoch_weighted_acc = accuracy_score(all_labels.cpu().numpy(), all_preds.cpu().numpy(), sample_weight=compute_sample_weight(class_weights, all_labels.cpu().numpy()))
            

            print('{} Loss: {:.4f} - Acc: {:.4f} - Weighted Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc, epoch_weighted_acc))

            # deep copy the model
            if phase == 'val' and epoch_weighted_acc > best_acc:
                best_acc = epoch_weighted_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_weighted_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

### 3.1 Training the Classifier Block of the Network

In [ ]:
# FREEZE THE WEIGHTS IN THE FEATURE EXTRACTION BLOCK OF THE NETWORK (I.E. RESNET BASE)
for param in model.features.parameters():
    param.requires_grad = False

In [ ]:
# TRAIN THE CLASSIFIER BLOCK OF THE MODEL (I.E TOP DENSE LAYERS)
model.to(device)
EPOCHS = 40
weight = torch.tensor([0.327, 0.586, 2.0, 2.572, 26.452, 15.0]).to(device)
criterion = nn.CrossEntropyLoss(weight=weight)
optimizer = optim.Adam(model.classifier.parameters(), lr=0.0005)

model, val_acc_history = train_model(model=model, dataloaders=data_loaders, criterion=criterion, optimizer=optimizer, num_epochs=EPOCHS)

In [ ]:
# SAVE CURRENT WEIGHTS OF THE MODEL (STAGE 1: FEATURE EXTRACTION)
torch.save(model.state_dict(), '../models/mvcnn_stage_1.pkl')

### 3.2 Fine-Tuning the Entire Network (Feature Extractor + Classifier Blocks)

In [ ]:
# UNFREEZE ALL THE WEIGHTS OF THE NETWORK
for param in model.parameters():
    param.requires_grad = True

In [ ]:
# FINE-TUNE THE ENTIRE MODEL (I.E FEATURE EXTRACTOR + CLASSIFIER BLOCKS) USING A VERY SMALL LEARNING RATE
EPOCHS = 20
criterion = nn.CrossEntropyLoss(weight=weight)
optimizer = optim.Adam(model.parameters(), lr=0.00005) # We use a smaller learning rate

model, val_acc_history = train_model(model=model, dataloaders=data_loaders, criterion=criterion, optimizer=optimizer, num_epochs=EPOCHS)

In [ ]:
# SAVE CURRENT WEIGHTS OF THE MODEL (STAGE 2: FINE-TUNING)
torch.save(model.state_dict(), '../models/mvcnn_stage_2.pkl')

In [ ]:
# FUNCTION TO GET PREDICTIONS FOR NEW IMAGES
def mvcnn_pred(_name, data_dir, model, device):
    transform = transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()])
    _fnames = glob.glob(data_dir + f'/{_name}_*.png')
    image = torch.stack([transform(Image.open(fname).convert('RGB')) for fname in _fnames]).unsqueeze(0)
    image = image.to(device)
    pred = torch.nn.functional.softmax(model(image)).argmax().item()
    return pred, {v:k for k,v in class_id_map.items()}[pred]



In [ ]:
# GET NEW PREDICTIONS FROM RAW IMAGES
plug_name = 'original_12'
mvcnn_pred(_name, '../data/processed', model, device)